In [2]:
path="D:\\data-mining\\images_compressed\\" #path to where original images are stored
path2="D:\\data-mining\\ClassifiedImages\\" #path where images after processed are stored
path3="D:\\data-mining\\test\\"
import pandas as pd

import matplotlib.pyplot as plt

In [3]:

ClothingData = pd.read_csv(r"D:\data-mining\images.csv") #dataset csv path

In [4]:
classlabel = []
i=0
for item_name in ClothingData.label: 
   if item_name=="Not sure":
      classlabel.append(0)
      
   elif item_name=="T-Shirt":
      classlabel.append(1)

   elif item_name=="Shoes":
      classlabel.append(2)
      
   elif item_name=="Shorts":
      classlabel.append(3)      

   elif item_name=="Shirt":
      classlabel.append(4)
      
   elif item_name=="Pants":
      classlabel.append(5)  
   
   elif item_name=="Skirt":
      classlabel.append(6)
      
   elif item_name=="Top":
      classlabel.append(7)
   
   elif item_name=="Outwear":
      classlabel.append(8)
      
   elif item_name=="Dress":
      classlabel.append(9)
   
   elif item_name=="Body":
      classlabel.append(10)
      
   elif item_name=="Longsleeve":
      classlabel.append(11)

   elif item_name=="Undershirt":
      classlabel.append(12)
      
   elif item_name=="Hat":
      classlabel.append(13)      

   elif item_name=="Polo":
      classlabel.append(14)
      
   elif item_name=="Blouse":
      classlabel.append(15)  
   
   elif item_name=="Hoodie":
      classlabel.append(16)
      
   elif item_name=="Skip":
      classlabel.append(17)
   
   elif item_name=="Blazer":
      classlabel.append(18)
   
   elif item_name=="Other":
      classlabel.append(19)
 
   

   

      

ClothingData['BinaryClassification'] = classlabel

ClothingData

image  sender_id     label   kids  \
0     4285fab0-751a-4b74-8e9b-43af05deee22        124  Not sure  False   
1     ea7b6656-3f84-4eb3-9099-23e623fc1018        148   T-Shirt  False   
2     00627a3f-0477-401c-95eb-92642cbe078d         94  Not sure  False   
3     ea2ffd4d-9b25-4ca8-9dc2-bd27f1cc59fa         43   T-Shirt  False   
4     3b86d877-2b9e-4c8b-a6a2-1d87513309d0        189     Shoes  False   
...                                    ...        ...       ...    ...   
5398  dfd4079d-967b-4b3e-8574-fbac11b58103        204    Shorts  False   
5399  befa14be-8140-4faf-8061-1039947e329d        204      Body   True   
5400  5379356a-40ee-4890-b416-2336a7d84061        310    Shorts  False   
5401  65507fb8-3456-4c15-b53e-d1b03bf71a59        204     Shoes  False   
5402  32b99302-cec7-4dec-adfa-3d4029674209        204     Skirt  False   

      BinaryClassification  
0                        0  
1                        1  
2                        0  
3                        1  
4                        2  
...                    ...  
5398                     3  
5399                    10  
5400                     3  
5401                     2  
5402                     6  

[5403 rows x 5 columns]

In [5]:

import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

import numpy as np

import pathlib

data_dir = path2
data_dir = pathlib.Path(data_dir)

image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

#split data to train and test

batch_size = 32
img_height = 180
img_width = 180

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)

407
Found 407 files belonging to 4 classes.
Using 326 files for training.
Found 407 files belonging to 4 classes.
Using 81 files for validation.
['Blazer', 'Hoodie', 'Skirt', 'Top']


In [6]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

for image_batch, labels_batch2 in val_ds:
  print(image_batch.shape)
  print(labels_batch2.shape)
  break


AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

normalization_layer = layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

(32, 180, 180, 3)
(32,)
(32, 180, 180, 3)
(32,)
0.0023972457 1.0


In [7]:
from keras import backend as K
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])


#model.compile(optimizer='adam',
#             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
 #            metrics=['accuracy'])

def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
  
def truepositive(y_true, y_pred):
  
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))

    return true_positives


model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer='Adam', metrics=['accuracy', precision, recall,truepositive])

#model.compile(optimizer='adam', loss=keras.losses.binary_crossentropy, metrics=['accuracy', tf.keras.metrics.SensitivityAtSpecificity(0.5)])

#view model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 180, 180, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 90, 90, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 90, 90, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 45, 45, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 45, 45, 64)        1

In [8]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/10
11/11 [==============================] - 3s 167ms/step - loss: 1.4157 - accuracy: 0.3650 - precision: 1.1213 - recall: 1.2419 - truepositive: 29.4545 - val_loss: 1.3391 - val_accuracy: 0.3704 - val_precision: 1.3283 - val_recall: 2.0139 - val_truepositive: 37.3333
Epoch 2/10
11/11 [==============================] - 2s 137ms/step - loss: 1.2595 - accuracy: 0.4264 - precision: 1.8948 - recall: 1.8928 - truepositive: 40.3636 - val_loss: 1.3218 - val_accuracy: 0.3704 - val_precision: 0.9737 - val_recall: 1.6667 - val_truepositive: 32.3333
Epoch 3/10
11/11 [==============================] - 1s 109ms/step - loss: 1.1756 - accuracy: 0.4448 - precision: 1.0772 - recall: 1.7394 - truepositive: 37.7273 - val_loss: 1.3073 - val_accuracy: 0.3580 - val_precision: 0.8080 - val_recall: 1.4702 - val_truepositive: 29.0000
Epoch 4/10
11/11 [==============================] - 2s 148ms/step - loss: 1.0328 - accuracy: 0.5368 - precision: 0.8771 - recall: 1.5502 - truepositive: 33.6364 - val_loss:

In [9]:

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)


In [10]:

# plt.figure(figsize=(8, 8))
# plt.subplot(1, 2, 1)
# plt.plot(epochs_range, acc, label='Training Accuracy')
# plt.plot(epochs_range, val_acc, label='Validation Accuracy')
# plt.legend(loc='lower right')
# plt.title('Training and Validation Accuracy')

# plt.subplot(1, 2, 2)
# plt.plot(epochs_range, loss, label='Training Loss')
# plt.plot(epochs_range, val_loss, label='Validation Loss')
# plt.legend(loc='upper right')
# plt.title('Training and Validation Loss')
# plt.show()

In [11]:
recall = history.history['recall']
val_recall = history.history['val_recall']
print("Recall Value:")
print(recall)
print(val_recall)

precision = history.history['precision']
val_precision = history.history['val_precision']
print("Precision Value")
print(precision)
print(val_precision)


tp = history.history['truepositive']
val_tp = history.history['val_truepositive']
print("True Positive Value: ")
print(tp)
print(val_tp)

Recall Value:
[1.241866946220398, 1.8928061723709106, 1.7394077777862549, 1.5501956939697266, 1.7064118385314941, 1.8501290082931519, 1.7206686735153198, 1.791158676147461, 1.8050559759140015, 1.7691227197647095]
[2.0138890743255615, 1.6666666269302368, 1.4702380895614624, 1.6130952835083008, 2.075396776199341, 1.8095237016677856, 1.9583333730697632, 2.0416667461395264, 1.9960317611694336, 2.047618865966797]
Precision Value
[1.1213077306747437, 1.8947844505310059, 1.077205777168274, 0.8770965337753296, 0.845950186252594, 0.8484963774681091, 0.7917295694351196, 0.8093149662017822, 0.8027322888374329, 0.7815141677856445]
[1.3283225297927856, 0.9736841320991516, 0.8079822659492493, 0.9722464084625244, 0.7775521278381348, 0.7682567238807678, 0.7616685032844543, 0.735079824924469, 0.7618309855461121, 0.7383996844291687]
True Positive Value: 
[29.454545974731445, 40.3636360168457, 37.727272033691406, 33.6363639831543, 37.272727966308594, 40.272727966308594, 38.09090805053711, 39.363636016845

In [12]:
img = tf.keras.utils.load_img(
    path3+'ee.jpg', target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)  
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

1/1 [==============================] - 0s 91ms/step
This image most likely belongs to Skirt with a 76.81 percent confidence.


In [13]:
from operator import index
import tensorflow as tf
import glob
import cv2
import os
from PIL import Image

data_dir = path3
#data_dir = pathlib.Path(data_dir)

image_data = []
label_data=[]

# iterate over files in
# that directory
for images in glob.iglob(f'{data_dir}/*'):
    #image = cv2.imread (images)
    
    #img = tf.keras.utils.load_img(images)
    img_array = tf.keras.utils.img_to_array(img)
    img_array=img_array/255
    image_data.append (img_array)
    i=0
   # for c in ClothingData.index: 
    #    if os.path.basename(images)==(ClothingData["image"].iloc[c]+".jpg"):
    #            label_data.append(ClothingData["BinaryClassification"].iloc[c])
        
    # check if the image ends with png
    





In [14]:
from keras_preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

test_generator = ImageDataGenerator()
test_data_generator = test_generator.flow_from_directory(
    path3, # Put your path here
     target_size=(img_width, img_height),
    batch_size=32,
    shuffle=False)
test_steps_per_epoch = np.math.ceil(test_data_generator.samples / test_data_generator.batch_size)

predictions = model.predict_generator(test_data_generator, steps=test_steps_per_epoch)
# Get most likely class
predicted_classes = np.argmax(predictions, axis=1)

true_classes = test_data_generator.classes
class_labels = list(test_data_generator.class_indices.keys())   


x=np.concatenate([test_data_generator.next()[0] for i in range(test_data_generator.__len__())])
y=np.concatenate([test_data_generator.next()[1] for i in range(test_data_generator.__len__())])

pixels = x.flatten()
pixels = x.flatten().reshape(203, 97200)
print (pixels.shape)
print(true_classes.shape)
print(x.shape)


report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)  



Found 203 images belonging to 4 classes.


c:\Users\mohaa\anaconda3\envs\meta\lib\site-packages\ipykernel_launcher.py:13: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  del sys.path[0]


(203, 97200)
(203,)
(203, 180, 180, 3)
              precision    recall  f1-score   support

      Blazer       0.98      0.95      0.96        55
      Hoodie       0.94      0.94      0.94        54
       Skirt       0.92      0.97      0.95        72
         Top       0.95      0.86      0.90        22

    accuracy                           0.95       203
   macro avg       0.95      0.93      0.94       203
weighted avg       0.95      0.95      0.95       203



In [15]:
confusion_matrix(true_classes, predicted_classes)



array([[52,  2,  1,  0],
       [ 1, 51,  2,  0],
       [ 0,  1, 70,  1],
       [ 0,  0,  3, 19]], dtype=int64)

In [16]:
from sklearn.metrics import plot_confusion_matrix

class estimator:
  _estimator_type = ''
  classes_=[]
  def __init__(self, model, classes):
    self.model = model
    self._estimator_type = 'classifier'
    self.classes_ = classes
  def predict(self, X):
    y_prob= self.model.predict(X)
    y_pred = y_prob.argmax(axis=1)
    return y_pred

classifier = estimator(model, class_names)


#figsize = (12,12)
#plot_confusion_matrix(estimator=classifier, X=normalizedtruth, y_true=normalizedpred, cmap='Blues', normalize='true', ax=plt.subplots(figsize=figsize)[1])



In [17]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(true_classes, predicted_classes)
# Normalise
cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(5,5))
sns.heatmap(cmn, annot=True, fmt='.2f', xticklabels=1, yticklabels=1)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show(block=False)

In [18]:
# from sklearn.model_selection import KFold, StratifiedKFold
# import tensorflow as tf
# #from tensorflow.keras.preprocessing.image import ImageDataGenerator

# idg = ImageDataGenerator(width_shift_range=0.1,
#                          height_shift_range=0.1,
#                          zoom_range=0.3,
#                          fill_mode='nearest',
#                          horizontal_flip = True,
#                          rescale=1./255)

# def get_model_name(k):
#     return 'model_'+str(k)+'.h5'

# VALIDATION_ACCURACY = []
# VALIDAITON_LOSS = []


# Y = ClothingData[['BinaryClassification']]

# kf = KFold(n_splits = 5)
                         
# skf = StratifiedKFold(n_split = 5, random_state = 7, shuffle = True) 


# save_dir = '/saved_models/'
# fold_var = 1

# for train_index, val_index in kf.split(np.zeros(n),Y):
# 	training_data = ClothingData.iloc[train_index]
# 	validation_data = ClothingData.iloc[val_index]
	
# 	train_data_generator = idg.flow_from_dataframe(training_data, directory = path2,
# 						       x_col = "filename", y_col = "label",
# 						       class_mode = "categorical", shuffle = True)
# 	valid_data_generator  = idg.flow_from_dataframe(validation_data, directory = path2,
# 							x_col = "filename", y_col = "label",
# 							class_mode = "categorical", shuffle = True)
	
# 	# CREATE NEW MODEL
# 	model = create_new_model()
# 	# COMPILE NEW MODEL
# 	model.compile(loss='categorical_crossentropy',
# 		      optimizer=opt,
# 		      metrics=['accuracy'])
	
# 	# CREATE CALLBACKS
# 	checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var), 
# 							monitor='val_accuracy', verbose=1, 
# 							save_best_only=True, mode='max')
# 	callbacks_list = [checkpoint]
# 	# There can be other callbacks, but just showing one because it involves the model name
# 	# This saves the best model
# 	# FIT THE MODEL
# 	history = model.fit(train_data_generator,
# 			    epochs=num_epochs,
# 			    callbacks=callbacks_list,
# 			    validation_data=valid_data_generator)
# 	#PLOT HISTORY
# 	#		:
# 	#		:
	
# 	# LOAD BEST MODEL to evaluate the performance of the model
# 	model.load_weights("/saved_models/model_"+str(fold_var)+".h5")
	
# 	results = model.evaluate(valid_data_generator)
# 	results = dict(zip(model.metrics_names,results))
	
# 	VALIDATION_ACCURACY.append(results['accuracy'])
# 	VALIDATION_LOSS.append(results['loss'])
	
# 	tf.keras.backend.clear_session()
	
# 	fold_var += 1

In [19]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
print(y_train)

[0 2 1 0 1 2 1 0 2 2 2 2 0 0 2 2 0 2 0 2 2 0 0 2 0 0 0 1 2 2 0 0 0 1 1 0 0
 1 0 2 1 2 1 0 2 0 2 0 0 2 0 2 1 1 1 2 2 1 1 0 1 2 2 0 1 1 1 1 0 0 0 2 1 2
 0]


In [20]:
from pickle import TRUE
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
X_train, X_test, y_train, y_test = train_test_split(pixels, true_classes, test_size=0.5, random_state=0)

gnb = GaussianNB()

Gaussian_Pred = gnb.fit(X_train, y_train).predict(X_test)




print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (y_test != Gaussian_Pred).sum()))
test=(y_test != Gaussian_Pred).sum()
accuracy=(test/X_test.shape[0])*100
print(accuracy)
report = classification_report(y_test, Gaussian_Pred, target_names=class_labels)
print(report)  
TP = 0
FP = 0
TN = 0
FN = 0

for i in range(len(Gaussian_Pred)): 
        if y_test[i]==Gaussian_Pred[i]==1:
           TP += 1
        if Gaussian_Pred[i]==1 and y_test[i]!=Gaussian_Pred[i]:
           FP += 1
        if y_test[i]==Gaussian_Pred[i]==0:
           TN += 1
        if Gaussian_Pred[i]==0 and y_test[i]!=Gaussian_Pred[i]:
           FN += 1
TPR = TP/(TP+FN) #sensitivigty
TNR = TN/(TN+FP) #Specificity
PPV = TP/(TP+FP) #Precision 

recall = TP / (TP + FN)
#auc = roc_auc_score(y_test, Gaussian_Pred)
#fpr, tpr, thresholds = roc_curve(y_test, Gaussian_Pred)
print("printing TP {}" .format(TP))
print("printing FP {}" .format(FP))
print("printing TN {}" .format(TN))
print("printing FN {}" .format(FN))
print("printing sensitivity {}" .format(TPR))
print("printing specificity {}" .format(TNR))
print("printing precision {}" .format(PPV))
print("printing recall {}" .format(recall))



Number of mislabeled points out of a total 102 points : 63
61.76470588235294
              precision    recall  f1-score   support

      Blazer       0.41      0.62      0.49        26
      Hoodie       0.52      0.44      0.48        27
       Skirt       0.35      0.18      0.24        38
         Top       0.20      0.36      0.26        11

    accuracy                           0.38       102
   macro avg       0.37      0.40      0.37       102
weighted avg       0.39      0.38      0.37       102

printing TP 12
printing FP 11
printing TN 16
printing FN 23
printing sensitivity 0.34285714285714286
printing specificity 0.5925925925925926
printing precision 0.5217391304347826
printing recall 0.34285714285714286


In [21]:
from pickle import TRUE
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
X_train, X_test, y_train, y_test = train_test_split(pixels, true_classes, test_size=0.5, random_state=0)

mnp = MultinomialNB()

nominal_pred = mnp.fit(X_train, y_train).predict(X_test)




print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (y_test != nominal_pred).sum()))
test=(y_test != nominal_pred).sum()
accuracy=(test/X_test.shape[0])*100
print(accuracy)
report = classification_report(y_test, nominal_pred, target_names=class_labels)
print(report)  
TP = 0
FP = 0
TN = 0
FN = 0

for i in range(len(nominal_pred)): 
        if y_test[i]==nominal_pred[i]==1:
           TP += 1
        if nominal_pred[i]==1 and y_test[i]!=nominal_pred[i]:
           FP += 1
        if y_test[i]==nominal_pred[i]==0:
           TN += 1
        if nominal_pred[i]==0 and y_test[i]!=nominal_pred[i]:
           FN += 1
TPR = TP/(TP+FN) #sensitivigty
TNR = TN/(TN+FP) #Specificity
PPV = TP/(TP+FP) #Precision 

recall = TP / (TP + FN)
#auc = roc_auc_score(y_test, nominal_pred)
#fpr, tpr, thresholds = roc_curve(y_test, nominal_pred)
print("printing TP {}" .format(TP))
print("printing FP {}" .format(FP))
print("printing TN {}" .format(TN))
print("printing FN {}" .format(FN))
print("printing sensitivity {}" .format(TPR))
print("printing specificity {}" .format(TNR))
print("printing precision {}" .format(PPV))
print("printing recall {}" .format(recall))



Number of mislabeled points out of a total 102 points : 59
57.84313725490197
              precision    recall  f1-score   support

      Blazer       0.41      0.62      0.49        26
      Hoodie       0.48      0.48      0.48        27
       Skirt       0.48      0.32      0.38        38
         Top       0.18      0.18      0.18        11

    accuracy                           0.42       102
   macro avg       0.39      0.40      0.38       102
weighted avg       0.43      0.42      0.41       102

printing TP 13
printing FP 14
printing TN 16
printing FN 23
printing sensitivity 0.3611111111111111
printing specificity 0.5333333333333333
printing precision 0.48148148148148145
printing recall 0.3611111111111111


In [22]:

from pickle import TRUE
from sklearn.naive_bayes import ComplementNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
X_train, X_test, y_train, y_test = train_test_split(pixels, true_classes, test_size=0.5, random_state=0)

cnp = ComplementNB()

complement_pred = cnp.fit(X_train, y_train).predict(X_test)




print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (y_test != complement_pred).sum()))
test=(y_test != complement_pred).sum()
accuracy=(test/X_test.shape[0])*100
print(accuracy)
report = classification_report(y_test, complement_pred, target_names=class_labels)
print(report)  
TP = 0
FP = 0
TN = 0
FN = 0

for i in range(len(complement_pred)): 
        if y_test[i]==complement_pred[i]==1:
           TP += 1
        if complement_pred[i]==1 and y_test[i]!=complement_pred[i]:
           FP += 1
        if y_test[i]==complement_pred[i]==0:
           TN += 1
        if complement_pred[i]==0 and y_test[i]!=complement_pred[i]:
           FN += 1
TPR = TP/(TP+FN) #sensitivigty
TNR = TN/(TN+FP) #Specificity
PPV = TP/(TP+FP) #Precision 

recall = TP / (TP + FN)
#auc = roc_auc_score(y_test, complement_pred)
#fpr, tpr, thresholds = roc_curve(y_test, complement_pred)
print("printing TP {}" .format(TP))
print("printing FP {}" .format(FP))
print("printing TN {}" .format(TN))
print("printing FN {}" .format(FN))
print("printing sensitivity {}" .format(TPR))
print("printing specificity {}" .format(TNR))
print("printing precision {}" .format(PPV))
print("printing recall {}" .format(recall))



Number of mislabeled points out of a total 102 points : 64
62.745098039215684
              precision    recall  f1-score   support

      Blazer       0.35      0.73      0.47        26
      Hoodie       0.39      0.52      0.44        27
       Skirt       0.50      0.13      0.21        38
         Top       0.00      0.00      0.00        11

    accuracy                           0.37       102
   macro avg       0.31      0.35      0.28       102
weighted avg       0.38      0.37      0.31       102

printing TP 14
printing FP 22
printing TN 19
printing FN 36
printing sensitivity 0.28
printing specificity 0.4634146341463415
printing precision 0.3888888888888889
printing recall 0.28


In [23]:

from pickle import TRUE
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
X_train, X_test, y_train, y_test = train_test_split(pixels, true_classes, test_size=0.5, random_state=0)

bnp = BernoulliNB()

bernoulli_pred = bnp.fit(X_train, y_train).predict(X_test)




print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (y_test != bernoulli_pred).sum()))
test=(y_test != bernoulli_pred).sum()
accuracy=(test/X_test.shape[0])*100
print(accuracy)
report = classification_report(y_test, bernoulli_pred, target_names=class_labels)
print(report)  
TP = 0
FP = 0
TN = 0
FN = 0

for i in range(len(bernoulli_pred)): 
        if y_test[i]==bernoulli_pred[i]==1:
           TP += 1
        if bernoulli_pred[i]==1 and y_test[i]!=bernoulli_pred[i]:
           FP += 1
        if y_test[i]==bernoulli_pred[i]==0:
           TN += 1
        if bernoulli_pred[i]==0 and y_test[i]!=bernoulli_pred[i]:
           FN += 1
print(TP)
print("printing TP {}" .format(TP))
print("printing FP {}" .format(FP))
print("printing TN {}" .format(TN))
print("printing FN {}" .format(FN))
TPR = TP/ (TP+FN) #sensitivigty
TNR = TN/( TN+FP) #Specificity
PPV = TP/ (TP+FP) #Precision 

recall = TP / (TP + FN)
#auc = roc_auc_score(y_test, bernoulli_pred)
#fpr, tpr, thresholds = roc_curve(y_test, bernoulli_pred)
print("printing TP {}" .format(TP))
print("printing FP {}" .format(FP))
print("printing TN {}" .format(TN))
print("printing FN {}" .format(FN))
print("printing sensitivity {}" .format(TPR))
print("printing specificity {}" .format(TNR))
print("printing precision {}" .format(PPV))
print("printing recall {}" .format(recall))



Number of mislabeled points out of a total 102 points : 64
62.745098039215684
              precision    recall  f1-score   support

      Blazer       0.00      0.00      0.00        26
      Hoodie       0.00      0.00      0.00        27
       Skirt       0.37      1.00      0.54        38
         Top       0.00      0.00      0.00        11

    accuracy                           0.37       102
   macro avg       0.09      0.25      0.14       102
weighted avg       0.14      0.37      0.20       102

0
printing TP 0
printing FP 0
printing TN 0
printing FN 0


c:\Users\mohaa\anaconda3\envs\meta\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mohaa\anaconda3\envs\meta\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mohaa\anaconda3\envs\meta\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ZeroDivisionError: division by zero

In [ ]:
from pickle import TRUE
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
X_train, X_test, y_train, y_test = train_test_split(pixels, true_classes, test_size=0.5, random_state=0)

cnx = CategoricalNB()

norm_pered = cnx.fit(X_train, y_train).predict(X_test)




print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (y_test != norm_pered).sum()))
test=(y_test != norm_pered).sum()
accuracy=(test/X_test.shape[0])*100
print(accuracy)
report = classification_report(y_test, norm_pered, target_names=class_labels)
print(report)  
TP = 0
FP = 0
TN = 0
FN = 0

for i in range(len(norm_pered)): 
        if y_test[i]==norm_pered[i]==1:
           TP += 1
        if norm_pered[i]==1 and y_test[i]!=norm_pered[i]:
           FP += 1
        if y_test[i]==norm_pered[i]==0:
           TN += 1
        if norm_pered[i]==0 and y_test[i]!=norm_pered[i]:
           FN += 1
TPR = TP/(TP+FN) #sensitivigty
TNR = TN/(TN+FP) #Specificity
PPV = TP/(TP+FP) #Precision 

recall = TP / (TP + FN)
#auc = roc_auc_score(y_test, norm_pered)
#fpr, tpr, thresholds = roc_curve(y_test, norm_pered)
print("printing TP {}" .format(TP))
print("printing FP {}" .format(FP))
print("printing TN {}" .format(TN))
print("printing FN {}" .format(FN))
print("printing sensitivity {}" .format(TPR))
print("printing specificity {}" .format(TNR))
print("printing precision {}" .format(PPV))
print("printing recall {}" .format(recall))



IndexError: index 250 is out of bounds for axis 1 with size 250